## import packages and dataset

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import statsmodels.stats.multitest as smm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [3]:
fifa = pd.read_csv("FIFA19data.csv", sep=r'\s*,\s*', engine='python')
fifa.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,5.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,5.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,5.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,4.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,4.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


# Prepare the Dataset 

In [4]:
# see all variable names
fifa.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Wage', 'International Reputation', 'Weak Foot', 'Skill Moves',
       'Work Rate', 'Body Type', 'Position', 'Contract Valid Until',
       'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
       'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl',
       'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance',
       'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving',
       'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes'],
      dtype='object')

In [5]:
# drop the variables not used in the analysis
fifa = fifa.drop('ID', 1)
fifa = fifa.drop('Name', 1)
fifa = fifa.drop('Nationality', 1)
fifa = fifa.drop('Club', 1)
fifa = fifa.drop('Value', 1)
fifa = fifa.drop('Wage', 1)
fifa = fifa.drop('Body Type', 1)
fifa = fifa.drop('Potential', 1)

In [6]:
# check if there is any NAs
fifa.isnull().values.any()

True

In [7]:
# replace all the NAs with the mode of the column
for col in fifa.columns:
    fifa[col].fillna(value=fifa[col].mode()[0], inplace=True)

In [7]:
# check the data type of the values in each column
for col in fifa:
    print(col, type(fifa[col][0]))

Age <class 'numpy.int64'>
Overall <class 'numpy.int64'>
International Reputation <class 'numpy.float64'>
Weak Foot <class 'numpy.float64'>
Skill Moves <class 'numpy.float64'>
Work Rate <class 'str'>
Position <class 'str'>
Contract Valid Until <class 'str'>
Crossing <class 'numpy.float64'>
Finishing <class 'numpy.float64'>
HeadingAccuracy <class 'numpy.float64'>
ShortPassing <class 'numpy.float64'>
Volleys <class 'numpy.float64'>
Dribbling <class 'numpy.float64'>
Curve <class 'numpy.float64'>
FKAccuracy <class 'numpy.float64'>
LongPassing <class 'numpy.float64'>
BallControl <class 'numpy.float64'>
Acceleration <class 'numpy.float64'>
SprintSpeed <class 'numpy.float64'>
Agility <class 'numpy.float64'>
Reactions <class 'numpy.float64'>
Balance <class 'numpy.float64'>
ShotPower <class 'numpy.float64'>
Jumping <class 'numpy.float64'>
Stamina <class 'numpy.float64'>
Strength <class 'numpy.float64'>
LongShots <class 'numpy.float64'>
Aggression <class 'numpy.float64'>
Interceptions <class 'num

In [8]:
# create a new column "Contract Length" with integer-type values
contractDate = []
for i in range(len(fifa['Contract Valid Until'])):
    try:
        contractDate.append((int(fifa['Contract Valid Until'][i])-2018))
    except:
        date_split = fifa['Contract Valid Until'][i].split("-")
        year = date_split[2]
        if len(year) == 4:
            contractDate.append(int(year)-2018)
        else:
            contractDate.append(int('20'+year)-2018)

fifa['Contract Length'] = contractDate

In [9]:
# create a copy the dataset, in which drop 'Contract Valid Until'
fifa_copy = fifa.copy()
fifa_copy = fifa_copy.drop('Contract Valid Until', 1)

In [10]:
# create dummy variables for each level within the categorical variables (non-ordinal)
factors = ['Work Rate', 'Position']

for var in factors:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(fifa_copy[var], prefix=var)
    fifa_copy = pd.concat([fifa_copy,cat_list], axis = 1)
    fifa_copy = fifa_copy.drop(var, 1)

In [11]:
# randomly split the data into train dataset (10%) and test dataset (90%)
X = fifa_copy.copy()
X = X.drop('Overall', 1)
Y = fifa_copy.copy()
Y = Y['Overall']
X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size=0.9, random_state=31)

# Q1: Basic Linear Model

## Full Model

In [12]:
lm1 = LinearRegression()
lm1.fit(X_train, y_train)
lm1_predictions = lm1.predict(X_test)
lm1_r2 = r2_score(y_test,lm1_predictions)
print("R-square of the model:", lm1_r2)
# adjusted R2
lm_train_score=lm1.score(X_train,y_train)
lm_test_score=lm1.score(X_test,y_test)
# print("training score:", lm_train_score)
lm_ra = 1-(1-lm_train_score)*((len(X_train)-1)/(len(X_train)-len(lm1.coef_)-1))
print("Adjusted R-square of the model:", lm_ra)

R-square of the model: 0.8911537742915339
Adjusted R-square of the model: 0.8969768998141294


In [13]:
X.shape
lm_coeff_used = np.sum(lm1.coef_!=0)
print("number of features used: ", lm_coeff_used)
# all features are used

number of features used:  75


## Cut Model

In [14]:
lm1_2 = sm.OLS(y_train, X_train)
results = lm1_2.fit()
# print(results.summary())

In [15]:
# use FDR- Benjamin-Hochberg procedure to select only significant variables
pvals = results.pvalues
rej, pval_corr = smm.multipletests(pvals, alpha = 0.1, method = "fdr_bh")[:2]
fdr_results = {'variables': X_train.columns, 'corrected_pvals': pval_corr, 'rej': rej}
fdr_results = pd.DataFrame(fdr_results)
# fdr_results

In [16]:
rej_var = fdr_results[fdr_results['rej'] == False]['variables']
print("number of featuers eliminated: ", len(rej_var))

X_train_red = X_train.copy()
X_test_red = X_test.copy()
for var in rej_var:
    X_train_red = X_train_red.drop(var, 1)
    X_test_red = X_test_red.drop(var, 1)

# print(X_train_red.shape)
# print(X_test_red.shape)

number of featuers eliminated:  18


In [17]:
lm2 = LinearRegression()
lm2.fit(X_train_red, y_train)
lm2_predictions = lm2.predict(X_test_red)
lm2_r2 = r2_score(y_test,lm2_predictions)
print("R-square of the model:", lm2_r2)
# adjusted R2
lm2_train_score=lm2.score(X_train_red,y_train)
lm2_test_score=lm2.score(X_test_red,y_test)
# print("training score:", lm_train_score)
lm2_ra = 1-(1-lm2_train_score)*((len(X_train_red)-1)/(len(X_train_red)-len(lm2.coef_)-1))
print("Adjusted R-square of the model:", lm2_ra)

R-square of the model: 0.8908207490817045
Adjusted R-square of the model: 0.897001422554668


#  Q2: CrossValidation

In [18]:
lm1_cv = cross_validate(lm1, X_train, y_train, cv=5)
lm1_scores = cross_val_score(lm1, X_train, y_train, cv=5)
lm1_scores
print("Accuracy for full model: %0.5f (+/- %0.5f)" % (lm1_scores.mean(), lm1_scores.std() * 2))

lm2_cv = cross_validate(lm2, X_train_red, y_train, cv=5)
lm2_scores = cross_val_score(lm2, X_train_red, y_train, cv=5)
lm2_scores
print("Accuracy for cut model: %0.5f (+/- %0.5f)" % (lm2_scores.mean(), lm2_scores.std() * 2))

# Since it has higher accuracy, we choose the cut model
# run the cross validation predict function to calculate OOS R^2:
lm2_cv_predictions = cross_val_predict(lm2, X_test_red, y_test, cv=5)
lm2_cv_r2 = r2_score(y_test,lm2_cv_predictions)
print("Average r2 after CV of lm2:", lm2_cv_r2)

Accuracy for full model: 0.88975 (+/- 0.02071)
Accuracy for cut model: 0.89139 (+/- 0.01823)
Average r2 after CV of lm2: 0.8938589150136973


# Q3: Lasso

In [19]:
# use Lasso regression to predict
lasso = Lasso()
lasso.fit(X_train,y_train)
lasso1_predictions = lasso.predict(X_test)

In [20]:
# result of the Lasso regression
lasso1_train_score=lasso.score(X_train,y_train)
lasso1_test_score=lasso.score(X_test,y_test)
lasso1_coeff_used = np.sum(lasso.coef_!=0)
r2_lasso1 = r2_score(y_test, lasso1_predictions)

# print("training score:", train_score)
# print("test score: ", test_score)
print("number of features used: ", lasso1_coeff_used)
print("test r2 score: ", r2_lasso1)

number of features used:  23
test r2 score:  0.8508221709052508


In [21]:
# test adjusted r2
lasso_ra = 1-(1-lasso1_test_score)*((len(X_test)-1)/(len(X_test)-lasso1_coeff_used-1))
print("Adjusted R-square of the model:", lasso_ra)

Adjusted R-square of the model: 0.85061248502435


# Q4a: Ridge

In [22]:
ridge = Ridge()
ridge.fit(X_train, y_train)
ridge_predictions = ridge.predict(X_test)

In [23]:
# result of the Ridge regression
ridge_train_score = ridge.score(X_train, y_train)
ridge_test_score = ridge.score(X_test, y_test)
ridge_coeff_used = np.sum(ridge.coef_ != 0)
r2_ridge = r2_score(y_test, ridge_predictions)

print("number of features used: ", ridge_coeff_used)
print("test r2 score: ", r2_ridge)

number of features used:  74
test r2 score:  0.8912216651356437


In [24]:
# test adjusted r2
ridge_ra = 1-(1-ridge_test_score)*((len(X_test)-1)/(len(X_test)-ridge_coeff_used-1))
print("Adjusted R-square of the model:", ridge_ra)

Adjusted R-square of the model: 0.8907281881383434


# Q4b: Log

Please refer to the write-up.

# Q5: Lasso Regression with Ideal Alpha

In [25]:
lasso2 = Lasso()

parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}

lasso_regressor = GridSearchCV(lasso2, parameters, cv = 5)

lasso_regressor.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3450.275075804783, tolerance: 7.077029670329669
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3486.1089424933693, tolerance: 7.259854670329671
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3361.8430056114153, tolerance: 7.131133997252747
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                   5, 10, 20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
# the ideal value of alpha 
print("the ideal value of alpha", lasso_regressor.best_params_)

the ideal value of alpha {'alpha': 0.01}


In [27]:
lasso2_coeff_used = np.sum(lasso_regressor.best_estimator_.coef_!=0)
print("the number of features used:", lasso2_coeff_used)

the number of features used: 52


In [28]:
lasso2_train_score = lasso_regressor.score(X_train,y_train)
lasso2_test_score = lasso_regressor.score(X_test,y_test)
lasso2_predictions = lasso_regressor.predict(X_test)
r2_lasso2 = r2_score(y_test, lasso2_predictions)
print("test r2 score:", r2_lasso2)

test r2 score: 0.8903681810518583


In [29]:
# adjusted test r2
lasso2_ra = 1-(1-lasso2_test_score)*((len(X_test)-1)/(len(X_test)-lasso2_coeff_used-1))
print("Adjusted R-square of the model:", lasso2_ra)

Adjusted R-square of the model: 0.8900191633840915


#  Q6: AIC BIC

In [30]:
# function of AIC
def AIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + 2*coeff_used

# function of AICc
def AICc(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + 2*coeff_used*(n/(n-coeff_used-1))

def BIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + np.log(n)*coeff_used

In [31]:
# AIC, AICc and BIC of simple linear full model
aic_lm1 = AIC(y_test, lm1_predictions, (len(X_test.columns)+1))
print(aic_lm1)

aicc_lm1 = AICc(y_test, lm1_predictions, (len(X_test.columns))+1)
print(aicc_lm1)

bic_lm1 = BIC(y_test, lm1_predictions, (len(X_test.columns)+1))
print(bic_lm1)

27107.75636311493
27108.473959681454
27693.278877972443


In [32]:
# AIC, AICc and BIC of simple linear cut model
aic_lm2 = AIC(y_test, lm2_predictions, (len(X_test_red.columns)+1))
print(aic_lm2)

aicc_lm2 = AICc(y_test, lm2_predictions, (len(X_test_red.columns))+1)
print(aicc_lm2)

bic_lm2 = BIC(y_test, lm2_predictions, (len(X_test_red.columns)+1))
print(bic_lm2)

27121.817378388096
27122.236535663942
27568.663508147776


In [33]:
# AIC, AICc and BIC of lasso model with ideal alpha
aic_lasso2 = AIC(y_test, lasso2_predictions, (lasso2_coeff_used+1))
print(aic_lasso2)

aicc_lasso2 = AICc(y_test, lasso2_predictions, (lasso2_coeff_used+1))
print(aicc_lasso2)

bic_lasso2 = BIC(y_test, lasso2_predictions, (lasso2_coeff_used+1))
print(bic_lasso2)

27179.60410311754
27179.954559249298
27587.929014794492
